# Multi Layer (Deep) Neural Network

## Load MNIST dataset

In [ ]:
from __future__ import print_function # for python 2 and 3 compatibility

import pandas as pd
import numpy as np

names = ['label']
feature_names = map(str, range(784))
names.extend(feature_names)

df_mnist_train = pd.read_csv('mnist_train.csv', header=None, nrows=1000, names=names)
df_mnist_test = pd.read_csv('mnist_test.csv', header=None, nrows=100, names=names)

# normalize pixel data (otherwise pure python implemention won't work)
max_pixel =  255
df_mnist_train.iloc[:, 1:] /= max_pixel
df_mnist_test.iloc[:, 1:] /= max_pixel


# select subset of features and labels 

X_train, y_train = df_mnist_train.iloc[:, 1:], df_mnist_train['label']
X_test, y_test = df_mnist_test.iloc[:, 1:], df_mnist_test['label']

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

## Part 1: tensorflow implementation

In [ ]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

# define model input
def input_fn(x, y):
    feature_cols = {name: tf.constant(x[name].values) for name in x.columns}
    label = tf.constant(y.values)
    return feature_cols, label


# define features
feat_cols = [tf.contrib.layers.real_valued_column(name) for name in X_train.columns]

# instantiate and fit model
model_2 = tf.contrib.learn.DNNClassifier(hidden_units=[100], feature_columns=feat_cols, n_classes=10) # need to specify n_classes
model_2.fit(input_fn=lambda: input_fn(X_train, y_train), steps=100)

# evaluate model
train_acc = model_2.evaluate(input_fn=lambda: input_fn(X_train, y_train), steps=1)['accuracy']
test_acc = model_2.evaluate(input_fn=lambda: input_fn(X_test, y_test), steps=1)['accuracy']
print('Train accuracy: {}'.format(train_acc))
print('Test accuracy: {}'.format(test_acc))

## Part 2: pure python implementation
### Adapted from Stanford CS231 softmax classifer tutorial

In [ ]:
# initialize parameters randomly
h = 100 # size of hidden layer
W = 0.01 * np.random.randn(D,h)
b = np.zeros((1,h))
W2 = 0.01 * np.random.randn(h,K)
b2 = np.zeros((1,K))

# some hyperparameters
step_size = 1e-0
reg = 1e-3 # regularization strength

# gradient descent loop
num_examples = X.shape[0]
for i in xrange(10000):
  
  # evaluate class scores, [N x K]
  hidden_layer = np.maximum(0, np.dot(X, W) + b) # note, ReLU activation
  scores = np.dot(hidden_layer, W2) + b2
  
  # compute the class probabilities
  exp_scores = np.exp(scores)
  probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]
  
  # compute the loss: average cross-entropy loss and regularization
  corect_logprobs = -np.log(probs[range(num_examples),y])
  data_loss = np.sum(corect_logprobs)/num_examples
  reg_loss = 0.5*reg*np.sum(W*W) + 0.5*reg*np.sum(W2*W2)
  loss = data_loss + reg_loss
  if i % 1000 == 0:
    print "iteration %d: loss %f" % (i, loss)
  
  # compute the gradient on scores
  dscores = probs
  dscores[range(num_examples),y] -= 1
  dscores /= num_examples
  
  # backpropate the gradient to the parameters
  # first backprop into parameters W2 and b2
  dW2 = np.dot(hidden_layer.T, dscores)
  db2 = np.sum(dscores, axis=0, keepdims=True)
  # next backprop into hidden layer
  dhidden = np.dot(dscores, W2.T)
  # backprop the ReLU non-linearity
  dhidden[hidden_layer <= 0] = 0
  # finally into W,b
  dW = np.dot(X.T, dhidden)
  db = np.sum(dhidden, axis=0, keepdims=True)
  
  # add regularization gradient contribution
  dW2 += reg * W2
  dW += reg * W
  
  # perform a parameter update
  W += -step_size * dW
  b += -step_size * db
  W2 += -step_size * dW2
  b2 += -step_size * db2

